In [ ]:
# Copyright 2022 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Notebook Setup

<table align="left">

  <td>
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/official/experiments/comparing_pipeline_runs.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Colab logo"> Run in Colab
    </a>
  </td>
  <td>
    <a href="https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/official/experiments/comparing_pipeline_runs.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo">
      View on GitHub
    </a>
  </td>
  <td>
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/official/experiments/comparing_pipeline_runs.ipynb">
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo">
      Open in Vertex AI Workbench
    </a>
  </td>                                                                                               
</table>

## Overview

Depending on the model life cycle of your data science team, you would like to experiment and track training Pipeline runs and its associated parameters. Then, you would to compare runs of these Pipelines to each others in order to figure out which is the best configuration generates the model you will register in the Vertex AI Model Registry.

### Dataset

The dataset used for this tutorial is the [Iris dataset](https://www.tensorflow.org/datasets/catalog/iris) from [TensorFlow Datasets](https://www.tensorflow.org/datasets/catalog/overview). This dataset does not require any feature engineering. The version of the dataset you will use in this tutorial is stored in a public Cloud Storage bucket. The trained model predicts the type of Iris flower species from a class of three species: setosa, virginica, or versicolor.


### Objective

This notebook shows two mehthods for tracking Vertex AI Experiments from Vertex Pipeline runs 

*   (1) Associate a `pipeline run` with an `experiment run`
*   (2) Manually log metadata, params, & metrics in pipeline components

The steps covered include:

* Formalize a training component
* Build a training pipeline
* Run several Pipeline jobs and log their results
* Compare different Pipeline jobs

#### Install additional packages

Install additional package dependencies not installed in your notebook environment, such as Vertex AI SDK and KFP SDK. Use the latest major GA version of each package.

In [1]:
import os

# The Vertex AI Workbench Notebook product has specific requirements
IS_WORKBENCH_NOTEBOOK = os.getenv("DL_ANACONDA_HOME")
IS_USER_MANAGED_WORKBENCH_NOTEBOOK = os.path.exists(
    "/opt/deeplearning/metadata/env_version"
)

# Vertex AI Notebook requires dependencies to be installed with '--user'
USER_FLAG = ""
if IS_WORKBENCH_NOTEBOOK:
    USER_FLAG = "--user"

!pip3 install {USER_FLAG} --force-reinstall git+https://github.com/sasha-gitg/python-aiplatform@main -q
!pip3 install {USER_FLAG} google-cloud-aiplatform[metadata] -q
!pip3 install {USER_FLAG} kfp -q

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tfx-bsl 1.9.0 requires google-api-python-client<2,>=1.7.11, but you have google-api-python-client 2.52.0 which is incompatible.
tfx-bsl 1.9.0 requires pyarrow<6,>=1, but you have pyarrow 8.0.0 which is incompatible.
tensorflow 2.9.0rc2 requires tensorboard<2.10,>=2.9, but you have tensorboard 2.8.0 which is incompatible.
tensorflow-transform 1.9.0 requires pyarrow<6,>=1, but you have pyarrow 8.0.0 which is incompatible.
tensorflow-serving-api 2.9.0 requires tensorflow<3,>=

#### Restart the kernel

After you install the additional packages, you need to restart the notebook kernel so it can find the packages.

In [2]:
# Automatically restart kernel after installs
import os

if not os.getenv("IS_TESTING"):
    # Automatically restart kernel after installs
    import IPython

    app = IPython.Application.instance()
    app.kernel.do_shutdown(True)

Set project ID

**If you don't know your project ID**, you may be able to get your project ID using `gcloud`.

In [1]:
import os

PROJECT_ID = "hybrid-vertex" # TODO:
REGION = "us-central1"   # TODO:

from datetime import datetime

TIMESTAMP = datetime.now().strftime("%Y%m%d%H%M%S")

Project ID:  hybrid-vertex


In [4]:
# If you are running this notebook in Colab, run this cell and follow the
# instructions to authenticate your GCP account. This provides access to your
# Cloud Storage bucket and lets you submit training jobs and prediction
# requests.

import os
import sys

# If on Vertex AI Workbench, then don't execute this code

IS_COLAB = False

if not os.path.exists("/opt/deeplearning/metadata/env_version") and not os.getenv(
    "DL_ANACONDA_HOME"
):
    if "google.colab" in sys.modules:

        IS_COLAB = True

        from google.colab import auth as google_auth

        google_auth.authenticate_user()

    # If you are running this notebook locally, replace the string below with the
    # path to your service account key and run this cell to authenticate your GCP
    # account.

    elif not os.getenv("IS_TESTING"):

        %env GOOGLE_APPLICATION_CREDENTIALS ''

In [5]:
BUCKET_NAME = "test-us-central"  # TODO:
BUCKET_URI = f"gs://{BUCKET_NAME}"

**Only if your bucket doesn't already exist**: Run the following cell to create your Cloud Storage bucket.

In [6]:
# ! gsutil mb -l $REGION -p $PROJECT_ID $BUCKET_URI

Finally, validate access to your Cloud Storage bucket by examining its contents:

In [7]:
! gsutil ls -al $BUCKET_URI

                                 gs://test-us-central/iris/
                                 gs://test-us-central/model/
                                 gs://test-us-central/pipelines/
                                 gs://test-us-central/tensorboard-logs/


#### Download training dataset


In [8]:
DATASET_URI = "gs://cloud-samples-data/ai-platform/iris"

In [9]:
!gsutil cp -r $DATASET_URI $BUCKET_URI

Copying gs://cloud-samples-data/ai-platform/iris/classification/evaluate.csv [Content-Type=text/csv]...
Copying gs://cloud-samples-data/ai-platform/iris/classification/train.csv [Content-Type=text/csv]...
Copying gs://cloud-samples-data/ai-platform/iris/iris_data.csv [Content-Type=application/octet-stream]...
Copying gs://cloud-samples-data/ai-platform/iris/iris_predict.csv [Content-Type=text/csv]...
- [4 files][  5.4 KiB/  5.4 KiB]                                                
==> NOTE: You are performing a sequence of gsutil operations that may
run significantly faster if you instead use gsutil -m cp ... Please
see the -m section under "gsutil help options" for further information
about when gsutil -m can be advantageous.

Copying gs://cloud-samples-data/ai-platform/iris/iris_target.csv [Content-Type=application/octet-stream]...
Copying gs://cloud-samples-data/ai-platform/iris/iris_test.csv [Content-Type=text/csv]...
Copying gs://cloud-samples-data/ai-platform/iris/iris_training.cs

## Import libraries and define constants

In [9]:
import logging
# General
import os
import time

logger = logging.getLogger("logger")
logging.basicConfig(level=logging.INFO)

import kfp.v2.compiler as compiler

# Pipeline Experiments
import kfp.v2.dsl as dsl

# Vertex AI
from google.cloud import aiplatform as vertex_ai
from kfp.v2.dsl import Metrics, Model, Output, component

**Create experiment for tracking training related metadata**
* First, create an experiment using the `init()` method and then initialize a run within the `experiment` using `start_run()`.

* `vertex_ai.init()` - Create an experiment instance
* `vertex_ai.start_run()` - Track a specific run within the experiment

# (1) Associate `pipeline run` with a Vertex Experiment

Add [PipelineJob](https://github.com/googleapis/python-aiplatform/blob/main/google/cloud/aiplatform/pipeline_jobs.py) metadata to `Vertex Experiment`
* experiment (Union[str, experiments_resource.Experiment]) - *optional*
* The Vertex AI experiment name or instance to associate to this PipelineJob.
* Metrics produced by the `PipelineJob` as `system.Metric Artifacts` will be associated as metrics to the current Experiment Run.
* Pipeline parameters will be associated as parameters to the current Experiment Run.


In [29]:
TIMESTAMP = datetime.now().strftime("%Y%m%d%H%M%S")
VERSION = 'v1'

# Experiments
TASK = "associated-experiment"
MODEL_TYPE = "xgboost"
EXPERIMENT_NAME = f"pipe-run-{TASK}-{VERSION}-{MODEL_TYPE}-{TIMESTAMP}"

# Pipeline
PIPELINE_URI = f"{BUCKET_URI}/pipelines"
TRAIN_URI = f"{BUCKET_URI}/iris/iris_data.csv"
LABEL_URI = f"{BUCKET_URI}/iris/iris_target.csv"
MODEL_URI = f"{BUCKET_URI}/model"

### Initialize Vertex AI SDK for Python

Initialize the Vertex AI SDK for Python for your project and corresponding bucket.

In [30]:
vertex_ai.init(project=PROJECT_ID, staging_bucket=BUCKET_URI)

### Set pre-built containers

Set the pre-built Docker container image for training and prediction.


For the latest list, see [Pre-built containers for training](https://cloud.google.com/ai-platform-unified/docs/training/pre-built-containers).


For the latest list, see [Pre-built containers for prediction](https://cloud.google.com/ai-platform-unified/docs/predictions/pre-built-containers).

In [31]:
TRAIN_IMAGE = vertex_ai.helpers.get_prebuilt_prediction_container_uri(
    framework="xgboost", framework_version="1.1", accelerator="cpu"
)

## Formalize your training as pipeline component


Before you start running your pipeline experiments, you have to formalize your training as pipeline component.

To do that, you will use the `kfp.v2.dsl.component` decorator to convert your training task into a pipeline component. 

In [32]:
@component(
    packages_to_install=[
        "numpy==1.18.5",
        "pandas==1.0.4",
        "scikit-learn==0.23.1",
        "xgboost==1.1.1",
    ]
)
def custom_trainer(
    train_uri: str,
    label_uri: str,
    max_depth: int,
    learning_rate: float,
    boost_rounds: int,
    model_uri: str,
    metrics: Output[Metrics],
    model_metadata: Output[Model],
):

    # import libraries
    import logging
    import uuid
    from pathlib import Path as path

    import pandas as pd
    import xgboost as xgb
    from sklearn.metrics import accuracy_score
    from sklearn.model_selection import train_test_split

    # variables
    gs_prefix = "gs://"
    gcsfuse_prefix = "/gcs/"
    train_path = train_uri.replace(gs_prefix, gcsfuse_prefix)
    label_path = label_uri.replace(gs_prefix, gcsfuse_prefix)
    model_path = model_uri.replace(gs_prefix, gcsfuse_prefix)

    def get_logger():
        """
        Get the logger
        """
        logger = logging.getLogger(__name__)
        logger.setLevel(logging.INFO)
        handler = logging.StreamHandler()
        handler.setFormatter(
            logging.Formatter("%(asctime)s - %(name)s - %(levelname)s - %(message)s")
        )
        logger.addHandler(handler)
        return logger

    def get_data(
        train_path: str, label_path: str
    ) -> (xgb.DMatrix, pd.DataFrame, pd.DataFrame):
        """
        Get the data
        Args:
            train_path: the path of the train data
            label_path: the path of the label data
        Returns:
            the train data and the label data
        """
        # Load data into pandas, then use `.values` to get NumPy arrays
        data = pd.read_csv(train_path).values
        labels = pd.read_csv(label_path).values

        # Convert one-column 2D array into 1D array for use with XGBoost
        labels = labels.reshape((labels.size,))
        train_data, test_data, train_labels, test_labels = train_test_split(
            data, labels, test_size=0.2, random_state=7
        )

        # Load data into DMatrix object
        dtrain = xgb.DMatrix(train_data, label=train_labels)
        return dtrain, test_data, test_labels

    def train_model(max_depth: int, eta: int, boost_rounds, dtrain: xgb.DMatrix):
        """
        Train the model
        Args:
            max_depth: the max depth of the model
            eta: the eta of the model
            boost_rounds: the boost rounds of the model
            dtrain: the train data
        Returns:
            the trained model
        """
        # Train XGBoost model
        param = {"max_depth": max_depth, "eta": eta}
        model = xgb.train(param, dtrain, num_boost_round=boost_rounds)
        return model

    def evaluate_model(model, test_data, test_labels):
        """
        Evaluate the model
        Args:
            model: the trained model
            test_data: the test data
            test_labels: the test labels
        Returns:
            the accuracy of the model
        """
        dtest = xgb.DMatrix(test_data)
        pred = model.predict(dtest)
        predictions = [round(value) for value in pred]
        # Evaluate predictions
        accuracy = accuracy_score(test_labels, predictions)
        return accuracy

    def save_model(model, model_path):
        """
        Save the model
        Args:
            model: the trained model
            model_path: the path of the model
        """
        model_id = str(uuid.uuid1())
        model_path = f"{model_path}/{model_id}/model.bst"
        path(model_path).parent.mkdir(parents=True, exist_ok=True)
        model.save_model(model_path)

    # Main ----------------------------------------------

    dtrain, test_data, test_labels = get_data(train_path, label_path)
    model = train_model(max_depth, learning_rate, boost_rounds, dtrain)
    accuracy = evaluate_model(model, test_data, test_labels)
    save_model(model, model_path)

    # Metadata ------------------------------------------
    metrics.log_metric("accurancy", accuracy)
    model_metadata.uri = model_uri

## Build a pipeline

### Define your workflow using Kubeflow Pipelines DSL package

> These parameters will automatically be logged to the `Experiment`

In [33]:
@dsl.pipeline(name=f"pipeline-run-{TASK}-{VERSION}")
def pipeline(
    train_uri: str,
    label_uri: str,
    max_depth: int,
    learning_rate: float,
    boost_rounds: int,
    model_uri: str,
):

    custom_trainer(
        train_uri, label_uri, max_depth, learning_rate, boost_rounds, model_uri
    )

#### Compile pipeline into a JSON file

In [34]:
compiler.Compiler().compile(pipeline_func=pipeline, package_path="pipeline.json")

### Submit and track pipeline runs

#### Submit Pipeline runs

Now that you have the pipeline, you define its training configuration depending on the defined parameters. Below you have an example and how you can submit several pipeline runs. 

In [35]:
runs = [
    {"max_depth": 4, "learning_rate": 0.2, "boost_rounds": 10},
    {"max_depth": 5, "learning_rate": 0.3, "boost_rounds": 20},
    {"max_depth": 3, "learning_rate": 0.1, "boost_rounds": 30},
    {"max_depth": 6, "learning_rate": 0.5, "boost_rounds": 40},
    {"max_depth": 5, "learning_rate": 0.4, "boost_rounds": 30},
]

In [36]:
for i, run in enumerate(runs):

    job = vertex_ai.PipelineJob(
        display_name=f"{EXPERIMENT_NAME}-pipeline-run-{i}",
        template_path="pipeline.json",
        pipeline_root=PIPELINE_URI,
        job_id=f"{EXPERIMENT_NAME}-pipeline-run-{i}",
        enable_caching=False,
        parameter_values={
            "train_uri": TRAIN_URI,
            "label_uri": LABEL_URI,
            "model_uri": MODEL_URI,
            **run,
        },
    )
    job.submit(experiment=EXPERIMENT_NAME)

Creating PipelineJob


INFO:google.cloud.aiplatform.pipeline_jobs:Creating PipelineJob


PipelineJob created. Resource name: projects/934903580331/locations/us-central1/pipelineJobs/pipe-run-associated-experiment-v1x-xgboost-20220729055009-pipeline-run-0


INFO:google.cloud.aiplatform.pipeline_jobs:PipelineJob created. Resource name: projects/934903580331/locations/us-central1/pipelineJobs/pipe-run-associated-experiment-v1x-xgboost-20220729055009-pipeline-run-0


To use this PipelineJob in another session:


...

INFO:google.cloud.aiplatform.pipeline_jobs:pipeline_job = aiplatform.PipelineJob.get('projects/934903580331/locations/us-central1/pipelineJobs/pipe-run-associated-experiment-v1x-xgboost-20220729055009-pipeline-run-4')


View Pipeline Job:
https://console.cloud.google.com/vertex-ai/locations/us-central1/pipelines/runs/pipe-run-associated-experiment-v1x-xgboost-20220729055009-pipeline-run-4?project=934903580331


INFO:google.cloud.aiplatform.pipeline_jobs:View Pipeline Job:
https://console.cloud.google.com/vertex-ai/locations/us-central1/pipelines/runs/pipe-run-associated-experiment-v1x-xgboost-20220729055009-pipeline-run-4?project=934903580331


Associating projects/934903580331/locations/us-central1/pipelineJobs/pipe-run-associated-experiment-v1x-xgboost-20220729055009-pipeline-run-4 to Experiment: pipe-run-associated-experiment-v1x-xgboost-20220729055009


INFO:google.cloud.aiplatform.metadata.experiment_resources:Associating projects/934903580331/locations/us-central1/pipelineJobs/pipe-run-associated-experiment-v1x-xgboost-20220729055009-pipeline-run-4 to Experiment: pipe-run-associated-experiment-v1x-xgboost-20220729055009


### Check Pipeline run states

Vertex AI SDK provides you `get_experiment_df` method to monitor the status of pipeline runs. You can use it either to return parameters and metrics of the Pipeline Runs in the Vertex AI Experiment or in combination with `get` method of `PipelineJob` to return the pipeline job in Vertex AI Pipeline.


In [37]:
# see state of all pipelineJob
vertex_ai.get_experiment_df(EXPERIMENT_NAME)

,experiment_name,run_name,run_type,state,param.boost_rounds,param.model_uri,param.max_depth,param.label_uri,param.learning_rate,param.train_uri,metric.accurancy
0,pipe-run-associated-experiment-v1x-xgboost-202...,pipe-run-associated-experiment-v1x-xgboost-202...,system.PipelineRun,RUNNING,30,gs://test-us-central/model,5,gs://test-us-central/iris/iris_target.csv,0.4,gs://test-us-central/iris/iris_data.csv,NaN
1,pipe-run-associated-experiment-v1x-xgboost-202...,pipe-run-associated-experiment-v1x-xgboost-202...,system.PipelineRun,RUNNING,40,gs://test-us-central/model,6,gs://test-us-central/iris/iris_target.csv,0.5,gs://test-us-central/iris/iris_data.csv,NaN
2,pipe-run-associated-experiment-v1x-xgboost-202...,pipe-run-associated-experiment-v1x-xgboost-202...,system.PipelineRun,COMPLETE,30,gs://test-us-central/model,3,gs://test-us-central/iris/iris_target.csv,0.1,gs://test-us-central/iris/iris_data.csv,0.9
3,pipe-run-associated-experiment-v1x-xgboost-202...,pipe-run-associated-experiment-v1x-xgboost-202...,system.PipelineRun,RUNNING,20,gs://test-us-central/model,5,gs://test-us-central/iris/iris_target.csv,0.3,gs://test-us-central/iris/iris_data.csv,NaN
4,pipe-run-associated-experiment-v1x-xgboost-202...,pipe-run-associated-experiment-v1x-xgboost-202...,system.PipelineRun,RUNNING,10,gs://test-us-central/model,4,gs://test-us-central/iris/iris_target.csv,0.2,gs://test-us-central/iris/iris_data.csv,NaN


In [38]:
while True:
    pipeline_experiments_df = vertex_ai.get_experiment_df(EXPERIMENT_NAME)
    if all(
        pipeline_state != "COMPLETE" for pipeline_state in pipeline_experiments_df.state
    ):
        print("Pipeline runs are still running...")
        if any(
            pipeline_state == "FAILED"
            for pipeline_state in pipeline_experiments_df.state
        ):
            print("At least one Pipeline run failed")
            break
    else:
        print("Pipeline experiment runs have completed")
        break
    time.sleep(60)

Pipeline experiment runs have completed


In [39]:
# Get the PipelineJob resource using the experiment run name
pipeline_experiments_df = vertex_ai.get_experiment_df(EXPERIMENT_NAME)
job = vertex_ai.PipelineJob.get(pipeline_experiments_df.run_name[0])
print(job.resource_name)
print(job._dashboard_uri())

projects/934903580331/locations/us-central1/pipelineJobs/pipe-run-associated-experiment-v1x-xgboost-20220729055009-pipeline-run-4
https://console.cloud.google.com/vertex-ai/locations/us-central1/pipelines/runs/pipe-run-associated-experiment-v1x-xgboost-20220729055009-pipeline-run-4?project=934903580331


# (2) Manually log Experiment metadata
* explictly start / stop experiment runs
* explicitly log params and metrics (as opposed to associating the pipeline run)

In [55]:
TIMESTAMP = datetime.now().strftime("%Y%m%d%H%M%S")
VERSION = 'v1'

# Experiments
TASK = "explicit-logging"
MODEL_TYPE = "xgboost"
EXPERIMENT_NAME = f"{TASK}-{VERSION}-{MODEL_TYPE}-{TIMESTAMP}"

# Pipeline
PIPELINE_URI = f"{BUCKET_URI}/pipelines"
TRAIN_URI = f"{BUCKET_URI}/iris/iris_data.csv"
LABEL_URI = f"{BUCKET_URI}/iris/iris_target.csv"
MODEL_URI = f"{BUCKET_URI}/model"

print(f"EXPERIMENT_NAME: {EXPERIMENT_NAME}")

EXPERIMENT_NAME: explicit-logging-v1-xgboost-20220729065531


#### See new code under `Variables`, `Experiments`, and `Experiment Run`

In [56]:
@component(
    packages_to_install=[
        "numpy==1.18.5",
        "pandas==1.0.4",
        "scikit-learn==0.23.1",
        "xgboost==1.1.1",
        "google-cloud-aiplatform",
    ]
)
def custom_trainer(
    project: str,
    location: str,
    staging_bucket: str,
    train_uri: str,
    label_uri: str,
    max_depth: int,
    learning_rate: float,
    epochs: int,
    batch_size: int,
    boost_rounds: int,
    model_uri: str,
    experiment_name: str,
    experiment_run: str,
    pipeline_name: str,
    metrics: Output[Metrics],
    model_metadata: Output[Model],
):

    # import libraries
    import logging
    import uuid
    from pathlib import Path as path

    import pandas as pd
    import xgboost as xgb
    from sklearn.metrics import accuracy_score
    from sklearn.model_selection import train_test_split
    import google.cloud.aiplatform as vertex_ai
    
    vertex_ai.init(project=project, location=location, staging_bucket=staging_bucket) # TODO

    # ====================================================
    # variables
    # ====================================================
    gs_prefix = "gs://"
    gcsfuse_prefix = "/gcs/"
    train_path = train_uri.replace(gs_prefix, gcsfuse_prefix)
    label_path = label_uri.replace(gs_prefix, gcsfuse_prefix)
    model_path = model_uri.replace(gs_prefix, gcsfuse_prefix)
    
    metaparams = {}
    metaparams["train_uri"] = train_uri
    metaparams["label_uri"] = label_uri
    metaparams["model_uri"] = model_uri
    metaparams["experiment_name"] = experiment_name
    metaparams["experiment_run"] = experiment_run
    
    hyperparams = {}
    hyperparams["epochs"] = epochs
    hyperparams["batch_size"] = batch_size
    hyperparams["learning_rate"] = learning_rate
    hyperparams["max_depth"] = max_depth
    hyperparams["boost_rounds"] = boost_rounds
    
    # ====================================================
    # Experiments
    # ====================================================
    
    EXPERIMENT_NAME = f"{experiment_name}"
    RUN_NAME = f"{experiment_run}"
    
    logging.info(f"Creating run: {RUN_NAME}; for experiment: {EXPERIMENT_NAME}")
    
    # Create experiment
    vertex_ai.init(experiment=EXPERIMENT_NAME)
    # vertex_ai.start_run(RUN_NAME,resume=True) # RUN_NAME
    
    with vertex_ai.start_run(RUN_NAME) as my_run:
        logging.info(f"logging metaparams")
        my_run.log_params(metaparams)
        
        logging.info(f"logging hyperparams")
        my_run.log_params(hyperparams)
    
    # ====================================================

    def get_logger():
        """
        Get the logger
        """
        logger = logging.getLogger(__name__)
        logger.setLevel(logging.INFO)
        handler = logging.StreamHandler()
        handler.setFormatter(
            logging.Formatter("%(asctime)s - %(name)s - %(levelname)s - %(message)s")
        )
        logger.addHandler(handler)
        return logger

    def get_data(
        train_path: str, label_path: str
    ) -> (xgb.DMatrix, pd.DataFrame, pd.DataFrame):
        """
        Get the data
        Args:
            train_path: the path of the train data
            label_path: the path of the label data
        Returns:
            the train data and the label data
        """
        # Load data into pandas, then use `.values` to get NumPy arrays
        data = pd.read_csv(train_path).values
        labels = pd.read_csv(label_path).values

        # Convert one-column 2D array into 1D array for use with XGBoost
        labels = labels.reshape((labels.size,))
        train_data, test_data, train_labels, test_labels = train_test_split(
            data, labels, test_size=0.2, random_state=7
        )

        # Load data into DMatrix object
        dtrain = xgb.DMatrix(train_data, label=train_labels)
        return dtrain, test_data, test_labels

    def train_model(max_depth: int, eta: int, boost_rounds, dtrain: xgb.DMatrix):
        """
        Train the model
        Args:
            max_depth: the max depth of the model
            eta: the eta of the model
            boost_rounds: the boost rounds of the model
            dtrain: the train data
        Returns:
            the trained model
        """
        # Train XGBoost model
        param = {"max_depth": max_depth, "eta": eta}
        model = xgb.train(param, dtrain, num_boost_round=boost_rounds)
        return model

    def evaluate_model(model, test_data, test_labels):
        """
        Evaluate the model
        Args:
            model: the trained model
            test_data: the test data
            test_labels: the test labels
        Returns:
            the accuracy of the model
        """
        dtest = xgb.DMatrix(test_data)
        pred = model.predict(dtest)
        predictions = [round(value) for value in pred]
        # Evaluate predictions
        accuracy = accuracy_score(test_labels, predictions)
        return accuracy

    def save_model(model, model_path):
        """
        Save the model
        Args:
            model: the trained model
            model_path: the path of the model
        """
        model_id = str(uuid.uuid1())
        model_path = f"{model_path}/{model_id}/model.bst"
        path(model_path).parent.mkdir(parents=True, exist_ok=True)
        model.save_model(model_path)

    # Main ----------------------------------------------

    dtrain, test_data, test_labels = get_data(train_path, label_path)
    model = train_model(max_depth, learning_rate, boost_rounds, dtrain)
    accuracy = evaluate_model(model, test_data, test_labels)
    save_model(model, model_path)

    # Metadata ------------------------------------------
    logging.info(f"logging metrics")
    metrics.log_metric("accuracy", accuracy)
    model_metadata.uri = model_uri
    
    # ====================================================
    # Experiment Run
    # ====================================================

    with vertex_ai.start_run(RUN_NAME,resume=True) as my_run:
        logging.info(f"logging accuracy to experiment run {RUN_NAME}")
        my_run.log_metrics({"accuracy": accuracy})
    
    # logging.info(f"Ending experiment run: {RUN_NAME}")
    # vertex_ai.end_run()

In [57]:
@dsl.pipeline(name=f"pipeline-run-{TASK}-{VERSION}")
def pipeline(
    train_uri: str,
    label_uri: str,
    max_depth: int,
    learning_rate: float,
    boost_rounds: int,
    model_uri: str,
    experiment_name: str,
    experiment_run: str,
    epochs: int,
    batch_size: int,
    pipeline_name: str,
    project: str,
    location: str,
    staging_bucket: str,
):

    custom_trainer(
        project=project,
        location=location,
        staging_bucket=staging_bucket,
        train_uri=train_uri, 
        label_uri=label_uri, 
        max_depth=max_depth, 
        learning_rate=learning_rate, 
        epochs=epochs, 
        pipeline_name=pipeline_name,
        batch_size=batch_size, 
        boost_rounds=boost_rounds, 
        model_uri=model_uri, 
        experiment_name=experiment_name,
        experiment_run=experiment_run,
    )

In [59]:
compiler.Compiler().compile(pipeline_func=pipeline, package_path="pipeline.json")

In [60]:
runs = [
    {"experiment_name": f'{EXPERIMENT_NAME}', "run_name": 'run-1', "max_depth": 4, "learning_rate": 0.2, "boost_rounds": 10, "epochs": 10, "batch_size": 200},
    {"experiment_name": f'{EXPERIMENT_NAME}', "run_name": 'run-2', "max_depth": 5, "learning_rate": 0.3, "boost_rounds": 20, "epochs": 12, "batch_size": 250},
    {"experiment_name": f'{EXPERIMENT_NAME}', "run_name": 'run-3', "max_depth": 3, "learning_rate": 0.1, "boost_rounds": 30, "epochs": 14, "batch_size": 300},
    {"experiment_name": f'{EXPERIMENT_NAME}', "run_name": 'run-4', "max_depth": 6, "learning_rate": 0.5, "boost_rounds": 40, "epochs": 16, "batch_size": 350},
    {"experiment_name": f'{EXPERIMENT_NAME}', "run_name": 'run-5', "max_depth": 5, "learning_rate": 0.4, "boost_rounds": 30, "epochs": 18, "batch_size": 400},
]

In [61]:
import google.cloud.aiplatform as vertex_ai
    
for i, run in enumerate(runs):

    job = vertex_ai.PipelineJob(
        display_name=f"{EXPERIMENT_NAME}-pipeline-run-{i}",
        template_path="pipeline.json",
        pipeline_root=f'{PIPELINE_URI}',
        job_id=f"{EXPERIMENT_NAME}-pipeline-run-{i}",
        enable_caching=False,
        parameter_values={
            'project': PROJECT_ID,
            'location': REGION,
            'staging_bucket': 'gs://test-us-central',
            "train_uri": TRAIN_URI,
            "label_uri": LABEL_URI,
            "model_uri": MODEL_URI,
            "experiment_name": run["experiment_name"],
            "experiment_run": run["run_name"],
            "max_depth": run["max_depth"],
            "learning_rate": run["learning_rate"],
            "boost_rounds": run["boost_rounds"],
            "epochs": run["epochs"],
            "batch_size": run["batch_size"],
            "pipeline_name": f"{EXPERIMENT_NAME}-pipeline-run-{i}",
        },
    )
    # job.submit(experiment=EXPERIMENT_NAME)
    job.submit()

Creating PipelineJob


INFO:google.cloud.aiplatform.pipeline_jobs:Creating PipelineJob


PipelineJob created. Resource name: projects/934903580331/locations/us-central1/pipelineJobs/explicit-logging-v1-xgboost-20220729065531-pipeline-run-0


INFO:google.cloud.aiplatform.pipeline_jobs:PipelineJob created. Resource name: projects/934903580331/locations/us-central1/pipelineJobs/explicit-logging-v1-xgboost-20220729065531-pipeline-run-0


To use this PipelineJob in another session:


...

INFO:google.cloud.aiplatform.pipeline_jobs:To use this PipelineJob in another session:


pipeline_job = aiplatform.PipelineJob.get('projects/934903580331/locations/us-central1/pipelineJobs/explicit-logging-v1-xgboost-20220729065531-pipeline-run-4')


INFO:google.cloud.aiplatform.pipeline_jobs:pipeline_job = aiplatform.PipelineJob.get('projects/934903580331/locations/us-central1/pipelineJobs/explicit-logging-v1-xgboost-20220729065531-pipeline-run-4')


View Pipeline Job:
https://console.cloud.google.com/vertex-ai/locations/us-central1/pipelines/runs/explicit-logging-v1-xgboost-20220729065531-pipeline-run-4?project=934903580331


INFO:google.cloud.aiplatform.pipeline_jobs:View Pipeline Job:
https://console.cloud.google.com/vertex-ai/locations/us-central1/pipelines/runs/explicit-logging-v1-xgboost-20220729065531-pipeline-run-4?project=934903580331


In [62]:
# see state of all pipelineJob
vertex_ai.get_experiment_df(EXPERIMENT_NAME)

,experiment_name,run_name,run_type,state,param.experiment_run,param.max_depth,param.train_uri,param.boost_rounds,param.learning_rate,param.epochs,param.model_uri,param.experiment_name,param.batch_size,param.label_uri,metric.accuracy
0,explicit-logging-v1-xgboost-20220729065531,run-4,system.ExperimentRun,COMPLETE,run-4,6.0,gs://test-us-central/iris/iris_data.csv,40.0,0.5,16.0,gs://test-us-central/model,explicit-logging-v1-xgboost-20220729065531,350.0,gs://test-us-central/iris/iris_target.csv,0.9
1,explicit-logging-v1-xgboost-20220729065531,run-3,system.ExperimentRun,COMPLETE,run-3,3.0,gs://test-us-central/iris/iris_data.csv,30.0,0.1,14.0,gs://test-us-central/model,explicit-logging-v1-xgboost-20220729065531,300.0,gs://test-us-central/iris/iris_target.csv,0.9
2,explicit-logging-v1-xgboost-20220729065531,run-1,system.ExperimentRun,COMPLETE,run-1,4.0,gs://test-us-central/iris/iris_data.csv,10.0,0.2,10.0,gs://test-us-central/model,explicit-logging-v1-xgboost-20220729065531,200.0,gs://test-us-central/iris/iris_target.csv,0.9


## Cleaning up

To clean up all Google Cloud resources used in this project, you can [delete the Google Cloud
project](https://cloud.google.com/resource-manager/docs/creating-managing-projects#shutting_down_projects) you used for the tutorial.

Otherwise, you can delete the individual resources you created in this tutorial

In [ ]:
# Delete the pipeline
for i in range(0, len(runs)):
    pipeline_job = vertex_ai.PipelineJob.get(pipeline_experiments_df.run_name[i])
    pipeline_job.delete()

In [ ]:
# Delete experiment
exp = vertex_ai.Experiment(EXPERIMENT_NAME)
exp.delete()

In [ ]:
delete_bucket = False

if delete_bucket or os.getenv("IS_TESTING"):
    ! gsutil rm -rf {BUCKET_URI}